## Difficult estimation

In [4]:
import numpy as np
import random
from pysat.solvers import Solver
from collections import deque
from scipy.ndimage import label
from scipy.spatial import distance
import matplotlib.pyplot as plt
import itertools 
import z3
import json

In [5]:
dataset = np.load("star_battle_dataset.npz")
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]

In [6]:
# Show 5 puzzles
for i in range(len(puzzles)):
    print(f"Showing puzzle {i+1}/{len(puzzles)}")
    print(puzzles[i])
    print(solutions[i])

Showing puzzle 1/5
[[0 0 0 0 9 9 9 1 1 1]
 [0 0 0 0 9 9 9 1 1 1]
 [0 2 2 0 9 1 1 1 1 1]
 [2 2 2 2 1 1 1 1 1 1]
 [5 2 5 8 7 7 7 1 1 4]
 [5 5 5 8 7 7 7 7 4 4]
 [5 5 5 8 7 7 7 6 4 4]
 [5 5 5 8 8 8 6 6 6 4]
 [5 5 5 5 6 6 6 6 6 3]
 [5 5 5 6 6 6 6 3 3 3]]
[[0 0 1 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1]
 [0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1]]
Showing puzzle 2/5
[[2 2 2 1 1 1 0 0 0 0]
 [2 2 2 1 1 1 0 0 0 0]
 [2 2 2 1 1 1 0 0 0 5]
 [7 1 1 1 1 1 0 0 5 5]
 [7 4 1 1 1 9 9 5 5 5]
 [7 4 8 8 8 3 9 9 5 5]
 [7 4 3 3 3 3 9 9 9 9]
 [7 4 4 3 3 3 9 9 9 9]
 [4 4 4 4 3 6 6 9 6 6]
 [4 4 4 4 4 6 6 6 6 6]]
[[0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 1]
 [1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0]
 [0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 1 0 0 0]]
Showing puzzle 3/5
[[1 0 0 0 0 4 4 4 4

In [ ]:
def classify_difficulty(puzzle, solution):
    #Classifies a Star Battle puzzle into Normal, Hard, or Extreme
    placed_stars = np.zeros_like(puzzle)  # Track placed stars
    definite_zeros = np.zeros_like(puzzle)  # Track definite non-star cells
    potential_stars = ()
    num_regions = 10


    if normal_solver(puzzle, solution, placed_stars, definite_zeros):
        return "Normal"
    elif hard_solver(puzzle, solution, placed_stars, definite_zeros):
        return "Hard"
    else:
        return "Extreme"

def normal_solver(puzzle, solution, placed_stars, definite_zeros):
    #attempts to solve using normal heuristics
    for _ in range(20):  # Limit iterations to simulate human solving
        made_progress = False

        made_progress |= apply_three_cell_straight_lines(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_straight_lines(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_t_shapes(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_l_shapes(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_pbdq_shapes(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_offset_stack(puzzle, solution, placed_stars, definite_zeros)        
        made_progress |= apply_simple_center_eliminations(puzzle, solution, placed_stars, definite_zeros)

        if np.array_equal(placed_stars, solution):
            return True

        if not made_progress:
            break  # No more easy deductions, try hard

    return np.array_equal(placed_stars, solution)  # Check if puzzle is fully solved

def hard_solver(puzzle, solution, placed_stars, definite_zeros):
    #attempts solve using normal and hard heuristics
    for _ in range(10):  # limited iterations
        made_progress = False

        made_progress |= apply_rule_of_crowding(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_rule_of_four_squares(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= apply_rule_of_clumps(puzzle, solution, placed_stars, definite_zeros)
        made_progress |= normal_solver(puzzle,solution, placed_stars, definite_zeros)

        if not made_progress:
            break  

    return np.array_equal(placed_stars, solution)  # Check if puzzle is fully solved



# specific heuristics functions 
def apply_three_cell_straight_lines(puzzle, placed_stars, definite_zeros):
    made_progress = False
    rows, cols = puzzle.shape

    # Iterate over each region number
    for region in np.unique(puzzle):
        # Find the cells belonging to this region
        region_cells = np.argwhere(puzzle == region)

        # Check if it's a 3-cell straight line (same row or same column)
        if len(region_cells) == 3:
            row_coords, col_coords = zip(*region_cells)  # Extract row and column indices

            # If the region forms a straight horizontal line
            if len(set(row_coords)) == 1 and len(set(col_coords)) == 3:
                row = row_coords[0]
                col_min, col_mid, col_max = sorted(col_coords)

                # Place stars at both ends
                if placed_stars[row, col_min] == 0 and placed_stars[row, col_max] == 0:
                    placed_stars[row, col_min] = 1
                    placed_stars[row, col_max] = 1
                    definite_zeros[row, col_mid] = 1  # Middle cell is a definite zero
                    made_progress = True

                    # Mark all other cells in the row as definite zeros
                    for c in range(cols):
                        if c not in [col_min, col_max]:
                            definite_zeros[row, c] = 1

            # If the region forms a straight vertical line
            elif len(set(col_coords)) == 1 and len(set(row_coords)) == 3:
                col = col_coords[0]
                row_min, row_mid, row_max = sorted(row_coords)

                # Place stars at both ends
                if placed_stars[row_min, col] == 0 and placed_stars[row_max, col] == 0:
                    placed_stars[row_min, col] = 1
                    placed_stars[row_max, col] = 1
                    definite_zeros[row_mid, col] = 1  # Middle cell is a definite zero
                    made_progress = True

                    # Mark all other cells in the column as definite zeros
                    for r in range(rows):
                        if r not in [row_min, row_max]:
                            definite_zeros[r, col] = 1

    return made_progress

def apply_straight_lines(puzzle, placed_stars, definite_zeros):
    made_progress = False
    num_rows, num_cols = puzzle.shape

    # Iterate over all regions
    unique_regions = np.unique(puzzle)

    for region in unique_regions:
        # Get all cells belonging to the current region
        region_cells = np.argwhere(puzzle == region)

        # Check if region forms a straight line
        row_coords, col_coords = zip(*region_cells)
        is_row = all(r == row_coords[0] for r in row_coords)
        is_col = all(c == col_coords[0] for c in col_coords)

        if is_row or is_col:
            region_size = len(region_cells)

            # Mark definite zeros in the entire row/column
            if is_row:
                r = row_coords[0]
                for c in range(num_cols):
                    if puzzle[r, c] != region:  # Only affect non-region cells
                        definite_zeros[r, c] = 1
            elif is_col:
                c = col_coords[0]
                for r in range(num_rows):
                    if puzzle[r, c] != region:
                        definite_zeros[r, c] = 1

            made_progress = True  # Changes were made

            # If exactly 3 cells, place stars at both ends
            if region_size == 3:
                placed_stars[region_cells[0][0], region_cells[0][1]] = 1  # First cell
                placed_stars[region_cells[2][0], region_cells[2][1]] = 1  # Last cell

                # Prevent stars from being overwritten as zeros
                definite_zeros[region_cells[0][0], region_cells[0][1]] = 0
                definite_zeros[region_cells[2][0], region_cells[2][1]] = 0

    return made_progress


def apply_t_shapes(puzzle, placed_stars, definite_zeros):
    # Identify and apply T-shape placement rules
    return False  

def apply_l_shapes(puzzle, placed_stars, definite_zeros):
    # Identify and apply L-shape placement rules
    return False  

def apply_pbdq_shapes(puzzle, placed_star, definite_zeross):
    # Identify and apply P/B/D/Q shape placement rules
    return False  

def apply_offset_stack(puzzle, placed_stars, definite_zeros):
    return False

def apply_simple_center_eliminations(puzzle, placed_stars, definite_zeros):
    # Apply eliminations based on 6-cell rectangles and 9-cell squares
    return False  

def apply_rule_of_crowding(puzzle, placed_stars, definite_zeros):
    # Implement Rule of Crowding logic
    return False  

def apply_rule_of_four_squares(puzzle, placed_stars, definite_zeros):
    # Implement Rule of Four-Squares logic
    return False  

def apply_rule_of_clumps(puzzle, placed_stars, definite_zeros):
    # Implement Rule of Clumps logic
    return False  



# testing for the functions


In [13]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
    placed_stars = np.zeros_like(puzzle)  # Track placed stars
    definite_zeros = np.zeros_like(puzzle)
    print(apply_straight_lines(puzzles[i], placed_stars, definite_zeros))

False
False
False
False
False


# classify

In [ ]:
# Classify each puzzle
difficulty_labels = [classify_difficulty(puzzles[i], solutions[i]) for i in range(len(puzzles))]

# Save labeled data
np.savez_compressed("star_battle_labeled_dataset.npz", puzzles=puzzles, solutions=solutions, labels=np.array(difficulty_labels))
print("Labeled dataset saved successfully!")

In [ ]:
# dataset = np.load("star_battle_dataset.npz")
# puzzles = dataset["puzzles"]
# solutions = dataset["solutions"]

difficulties = []
for i in range(len(puzzles)):
    difficulty_score = classify_difficulty(puzzles[i], solutions[i])
    
    

# Save difficulty labels
print(difficulties)
# np.savez_compressed("star_battle_dataset_with_difficulty.npz", puzzles=puzzles, solutions=solutions, difficulties=difficulties)
# print("Dataset with difficulty scores saved!")


['Hard', 'Hard', 'Hard', 'Hard', 'Hard']
Dataset with difficulty scores saved!
